In [1]:
from functools import wraps
from inspect import signature

    
def param_info(func):
    sig = signature(func)
    for param in sig.parameters.values():
        print(param.name)
        print(' -', param.default)
        print(' -', param.kind)


def safe_param(func):
    ok_args = False
    ok_kwargs = False
    
    list_params = []
    keyword_params = set()
    
    sig = signature(func)
    for param in sig.parameters.values():
        if param.kind == param.VAR_POSITIONAL:
            ok_args = True
        if param.kind == param.VAR_KEYWORD:
            ok_kwargs = True
            
        if param.kind in [param.POSITIONAL_OR_KEYWORD]:
            list_params.append(param.name)
        if param.kind in [param.POSITIONAL_OR_KEYWORD, param.KEYWORD_ONLY]:
            keyword_params.add(param.name)
            
    def get_default_value(param_name):
        original = sig.parameters[param_name]
        no_default = original.default is original.empty
        return None if original.default is original.empty else original.default
            
    @wraps(func)
    def wrap(*args, **kwargs):
        if not ok_args:
            args = args[:len(list_params)]
        
        if not ok_kwargs:
            temp = {k: v for k, v in kwargs.items() if k in keyword_params}
            kwargs = temp
        
        if len(args) < len(list_params):
            not_set_list_params = list_params[len(args):]
            for param in not_set_list_params:
                if param in kwargs:
                    continue
                    
                kwargs[param] = get_default_value(param)
        
        not_set_keyword_params = keyword_params - set(list_params) - set(kwargs.keys())
        for param in not_set_keyword_params:
            kwargs[param] = get_default_value(param)
        
        return func(*args, **kwargs)
    return wrap

In [2]:
def sample(a, b, *args, c, **kwargs):
    print('a', a)
    print('b', b)
    print('c', c)
    print('args', args)
    print('kwargs', kwargs)
    
safe_sample = safe_param(sample)
        
print('<sample>')
param_info(sample)
print('\n<safe_sample>')
param_info(safe_sample)

<sample>
a
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
b
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
args
 - <class 'inspect._empty'>
 - VAR_POSITIONAL
c
 - <class 'inspect._empty'>
 - KEYWORD_ONLY
kwargs
 - <class 'inspect._empty'>
 - VAR_KEYWORD

<safe_sample>
a
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
b
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
args
 - <class 'inspect._empty'>
 - VAR_POSITIONAL
c
 - <class 'inspect._empty'>
 - KEYWORD_ONLY
kwargs
 - <class 'inspect._empty'>
 - VAR_KEYWORD


In [3]:
safe_sample(1,2,3,4,5,6,7, d=10)
sample(1,2,3,4,5,6,7, d=10)

a 1
b 2
c None
args (3, 4, 5, 6, 7)
kwargs {'d': 10}


TypeError: sample() missing 1 required keyword-only argument: 'c'

In [4]:
def sample2(a, b, *, c):
    print('a', a)
    print('b', b)
    print('c', c)
    
safe_sample2 = safe_param(sample2)

safe_sample2(1,2,3,4,5,6,7, d=10)
sample2(1,2,3,4,5,6,7, d=10)

a 1
b 2
c None


TypeError: sample2() got an unexpected keyword argument 'd'

In [7]:
def safe_param(default=None):
    def deco(func):
        ok_args = False
        ok_kwargs = False

        list_params = []
        keyword_params = set()

        sig = signature(func)
        for param in sig.parameters.values():
            if param.kind == param.VAR_POSITIONAL:
                ok_args = True
            if param.kind == param.VAR_KEYWORD:
                ok_kwargs = True

            if param.kind in [param.POSITIONAL_OR_KEYWORD]:
                list_params.append(param.name)
            if param.kind in [param.POSITIONAL_OR_KEYWORD, param.KEYWORD_ONLY]:
                keyword_params.add(param.name)

        def get_default_value(param_name):
            original = sig.parameters[param_name]
            no_default = original.default is original.empty
            return default if original.default is original.empty else original.default

        @wraps(func)
        def wrap(*args, **kwargs):
            if not ok_args:
                args = args[:len(list_params)]

            if not ok_kwargs:
                temp = {k: v for k, v in kwargs.items() if k in keyword_params}
                kwargs = temp

            if len(args) < len(list_params):
                not_set_list_params = list_params[len(args):]
                for param in not_set_list_params:
                    if param in kwargs:
                        continue

                    kwargs[param] = get_default_value(param)

            not_set_keyword_params = keyword_params - set(list_params) - set(kwargs.keys())
            for param in not_set_keyword_params:
                kwargs[param] = get_default_value(param)

            return func(*args, **kwargs)
        return wrap
    return deco

In [10]:
@safe_param('is default')
def sample3(a, b, *args, c, **kwargs):
    print('a', a)
    print('b', b)
    print('c', c)
    print('args', args)
    print('kwargs', kwargs)
    
sample3(1,2,3,4,5,6,7, d=10)

a 1
b 2
c is default
args (3, 4, 5, 6, 7)
kwargs {'d': 10}
